# Building a Drake scene with `airo-drake` 🏗️️

Ready to set up your robot scene in Drake? This notebook will show you how `airo-drake` makes the process a breeze! ️🏖️ `airo-drake` is designed to streamline common tasks, but still give you full flexibility to customize your Drake diagrams.

For comparison, we've included a more "native" way to build a similar Drake scene that directly uses the models from [`airo-models`](https://github.com/airo-ugent/airo-models).

## 1. The `airo-drake` way 🚀

Run the cell below and open click on the link to open the visualizer.

In [ ]:
import numpy as np
from pydrake.planning import RobotDiagramBuilder
from airo_drake import add_manipulator, add_floor, add_meshcat, finish_build

robot_diagram_builder = RobotDiagramBuilder() 

meshcat = add_meshcat(robot_diagram_builder)
arm_index, gripper_index = add_manipulator(robot_diagram_builder, "ur5e", "robotiq_2f_85")
add_floor(robot_diagram_builder)

robot_diagram, context = finish_build(robot_diagram_builder, meshcat)
del robot_diagram_builder # no longer needed

### 1.1 Understanding Drake's building blocks 🧩

Drake doesn't have a single "Scene" class. Instead, here are the essential components you'll want to keep around:


  - 🤖 `RobotDiagram`: The core of your scene. It houses the Drake systems and is crucial for tasks like collision checking and time parameterization.
  - 🐈 `MeshCat`: The browser-based visualizer (e.g., at `localhost:7000`). Our `add_meshcat()` function creates it along with a `MeshCatVisualizer` that syncs the last published context. 
  - 🗂️ `ModelInstanceIndices`: The model identifiers, allowing you to track and manipulate the models in your scene.

### 1.2 Helping you stay organized 🖼️

For ease of use, we offer the optional `SingleArmScene` and `DualArmScene` dataclasses to group these components. Feel free to use them. It's not required, all our function operate on the individual objects directly – the choice is yours!

In [ ]:
from airo_drake import SingleArmScene

scene = SingleArmScene(robot_diagram, arm_index, gripper_index, meshcat)
scene

> 💡 **Tip:** Editing Drake diagrams (e.g. adding new obstacles) after finalization is not trivial. It's often easier to create an entirely new scene and treat scenes as disposable 🗑️. 

## 2. Basic usage 📖

More advanced usage and visualizations are shown in the next notebook. Here we'll simply show how you can get some basic information and how you can change the pose of the robot arm.

In [ ]:
plant = scene.robot_diagram.plant()
plant

In [ ]:
plant.num_positions(scene.arm_index)

In [ ]:
plant_context = plant.GetMyContextFromRoot(context)
plant.GetPositions(plant_context, scene.arm_index)

In [ ]:
joints_new = np.deg2rad([0, -45, -90, -90, 90, 0])

plant.SetPositions(plant_context, scene.arm_index, joints_new)
scene.robot_diagram.ForcedPublish(context) # updates the meshcat visualization

In [ ]:
plant.GetPositions(plant_context, scene.arm_index)

## 3. The long way ⛰️

The code below does almost the same thing as the code above (i.e. without `airo-drake`). 

It's more hackable, but you lose a lot of convenience and some useful defaults we set with `airo-drake`

In [ ]:
import numpy as np
import airo_models
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.geometry import Meshcat
from pydrake.geometry import MeshcatVisualizer

robot_diagram_builder = RobotDiagramBuilder()
scene_graph = robot_diagram_builder.scene_graph()
plant = robot_diagram_builder.plant()
builder = robot_diagram_builder.builder()
plant = robot_diagram_builder.plant()
parser = robot_diagram_builder.parser()
parser.SetAutoRenaming(True)

meshcat = Meshcat()
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

# Load URDF files
ur5e_urdf_path = airo_models.get_urdf_path("ur5e")
robotiq_urdf_path = airo_models.get_urdf_path("robotiq_2f_85")

floor_thickness = 0.2
floor_urdf_path = airo_models.box_urdf_path((3.0, 3.0, floor_thickness), "floor")

arm_index = parser.AddModels(ur5e_urdf_path)[0]
gripper_index = parser.AddModels(robotiq_urdf_path)[0]
floor_index = parser.AddModels(floor_urdf_path)[0]

# Weld some frames together
world_frame = plant.world_frame()
floor_frame = plant.GetFrameByName("base_link", floor_index)
arm_frame = plant.GetFrameByName("base_link", arm_index)
arm_tool_frame = plant.GetFrameByName("tool0", arm_index)
gripper_frame = plant.GetFrameByName("base_link", gripper_index)

arm_transform = RigidTransform(rpy=RollPitchYaw([0, 0, np.pi]), p=[0, 0, 0])
floor_transform = RigidTransform(p=[0, 0, -floor_thickness / 2])

X_URTOOL0_ROBOTIQ = RigidTransform(rpy=RollPitchYaw([0, 0, np.pi / 2]), p=[0, 0, 0])

plant.WeldFrames(world_frame, arm_frame, arm_transform)
plant.WeldFrames(arm_tool_frame, gripper_frame, X_URTOOL0_ROBOTIQ)
plant.WeldFrames(world_frame, floor_frame, floor_transform)

# Finishing and visualizing
diagram = robot_diagram_builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
diagram.ForcedPublish(context)